<h1>PTA Data Analysis with enterprise</h1>

This tutorial will walk you through the basics of running a gravitational wave search on pulsar timing array data using enterprise.

In [ ]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'

from __future__ import division

import numpy as np
import glob, os, json
import matplotlib.pyplot as plt

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

The first step is to load the data. Pulsar timing data is stored as par files, which contain a list of the pulsar's parameters and their uncertainties, and tim files, which contain the pulse times of arrival. Here we initialize all of the pulsar objects from the par and tim files (this will take a while).

In this tutorial, we will only analyze a single pulsar, J1713+0747.

In [ ]:
datadir = '/home/sarah.vigeland/nanograv_data/11yr/partim/'

In [ ]:
parfiles = sorted(glob.glob(datadir + 'J1713*.par'))
timfiles = sorted(glob.glob(datadir + 'J1713*.tim'))

psrs = []
for p, t in zip(parfiles, timfiles):
    psrname = parfiles[0].split('/')[-1].split('_')[0]
    print 'Loading {0}'.format(psrname)
    psr = Pulsar(p, t, ephem='DE436', clk='BIPM2015')
    psrs.append(psr)

First, let's plot the residuals for this pulsar.

In [ ]:
plt.errorbar(psrs[0].toas, psrs[0].residuals, ls='', marker='.', yerr=psrs[0].toaerrs)
plt.ylabel('Residuals');
plt.xlabel('MJD');

First, let's add the white noise. There are three terms describing white noise: EFAC, which is a linear scaling of the TOA uncertainties; EQUAD, which is white noise added in quadrature to the TOA uncertainties; and ECORR, which is correlated over a single observation but uncorrelated between different observations.

In our analysis, we will fix the white noise to the maximum-likelihood values, which are found from noise analyses run on each pulsar. For now we will initialize the white noise parameters as constants. Later, we will read in the white noise values from a text file.

In [ ]:
# white noise parameters
# here we define them as constants, later we will input the values after the model is initialized
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# there will be separate white noise parameters for each observing backend
# since NANOGrav began taking data, there have been two generations of backends
# (ASP and PUPPI at Arecibo, GASP and GUPPI at Green Bank)
selection = selections.Selection(selections.by_backend)

# white noise signals
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = gp_signals.EcorrBasisModel(log10_ecorr=ecorr, selection=selection)

Now we add in the red noise. We model the red noise as a power law with two parameters: an amplitude and spectral index.

In [ ]:
# red noise parameters
log10_A = parameter.Uniform(-18, -11)
gamma = parameter.Uniform(0, 7)

# define powerlaw PSD and red noise signal
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(pl, components=30, name='rn')

We also include contributions from the pulsar's timing model.

In [ ]:
# linearized timing model
tm = gp_signals.TimingModel(use_svd=False)

Finally, we define the GW background. We model the background as a power law with fixed spectral index and an amplitude parameter. You can also do runs where you vary the spectral index and the amplitude simultaneously.

In [ ]:
# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_Agw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')

# gwb (no spatial correlations)
cpl = utils.powerlaw(log10_A=log10_Agw, gamma=gamma_gw)
gwb = gp_signals.FourierBasisGP(spectrum=cpl, components=30, name='gwb')

Now we put together all of the signals to define the model.

In [ ]:
# full signal
s = ef + eq + ec + rn + tm + gwb

Since we are only analzying a single pulsar, we will not include ephemeris modeling into the model. If you want to include it, set `bayesephem = True`.

In [ ]:
# if you want to include ephemeris modeling, set bayesephem = True
bayesephem = False
if bayesephem:
    s += deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

Now we initalize the PTA object, and fix the pulsars' white noise parameters to their maximum-likelihood values.

In [ ]:
# initialize PTA
model = [s(psr) for psr in psrs]
pta = signal_base.PTA(model)

We get the pulsars' white noise parameters from their noise files.

In [ ]:
def get_noise_from_pal2(noisefile):
    psrname = noisefile.split('/')[-1].split('_noise.txt')[0]
    fin = open(noisefile, 'r')
    lines = fin.readlines()
    params = {}
    for line in lines:
        ln = line.split()
        if 'efac' in line:
            par = 'efac'
            flag = ln[0].split('efac-')[-1]
        elif 'equad' in line:
            par = 'log10_equad'
            flag = ln[0].split('equad-')[-1]
        elif 'jitter_q' in line:
            par = 'log10_ecorr'
            flag = ln[0].split('jitter_q-')[-1]
#            flag = 'ecorr_{0}'.format(ln[0].split('jitter_q-')[-1])
        elif 'RN-Amplitude' in line:
            par = 'log10_A'
            flag = ''
        elif 'RN-spectral-index' in line:
            par = 'gamma'
            flag = ''
        else:
            break
        if flag:
            name = [psrname, flag, par]
        else:
            name = [psrname, par]
        pname = '_'.join(name)
        params.update({pname: float(ln[1])})
        
    return params

In [ ]:
noisedir = '/home/sarah.vigeland/nanograv_data/11yr/noisefiles/'
noisefiles = glob.glob(noisedir + '*_noise.txt')

setpars = {}
for nf in noisefiles:
    setpars.update(get_noise_from_pal2(nf))

pta.set_default_params(setpars)

Here we draw the initial values for the parameters.

This model only contains three parameters since we are only analyzing one pulsar, and we are not including `BayesEphem`. In general, if you run a gravitational wave search with fixed spectral index and without `BayesEphem`, you will have 2N + 1 parameters, where N is the number of pulsars in the PTA.

In [ ]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

print ndim

In [ ]:
pta.params

We need to set up a few other things before running the sampler. The sampler uses Adaptive Metropolis sampling, which uses the covariance matrix to determine the step sizes for the parameters. We initialize the covariance matrix as a diagonal matrix with the same value in each diagonal element. As the code runs, the covariance matrix will be updated based on the samples in the chain.

We also need to set up parameter groups, which determine which parameters should be jumped in simultaneously. For this run, we only have three groups: one containing all of the parameters, one containing both of the red noise parameters, and one containing the GWB amplitude.

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

# set up jump groups by red noise groups 
ndim = len(x0)
groups  = [range(0, ndim)]
groups.extend(map(list, zip(range(0,ndim,2), range(1,ndim,2))))
groups.extend([[ndim-1]])
print groups

In [ ]:
outDir = 'chains/'

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups, 
                 outDir='chains/', resume=True)

In [ ]:
# sampler for N steps
N = int(1e6)
x0 = np.hstack(p.sample() for p in pta.params)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )